# Model 3: Transformer Model

In [ ]:
import sys; sys.path.insert(0, '/mnt/src')

In [ ]:
from models.transformer import TransformerEncoderModel, train

In [ ]:
feature_columns = [
    'left_boom_base_yaw_joint', 
    'left_boom_base_pitch_joint',
    'left_boom_main_prismatic_joint',
    'left_boom_second_roll_joint',
    'left_boom_second_yaw_joint',
    'left_boom_top_pitch_joint',
    'left_boom_ee_joint',
    'cable1_property(length,youngsmodule(bend,twist))',
    'cable2_property(length,youngsmodule(bend,twist))',
    'cable3_property(length,youngsmodule(bend,twist))'
]

label_features = [
    ('cable1_lowest_point', np.array([1], dtype=np.int64)),
    ('cable2_lowest_point', np.array([1], dtype=np.int64)),
    ('cable3_lowest_point', np.array([1], dtype=np.int64))
]

normalized_features = [
    ('cable1_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable2_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable3_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64))
]

In [ ]:
data_folder = Path("/mnt/data").absolute()
train_set, _, validation_set, _ = read_trajectory_datasets(data_folder, 0.8, 0, 0.2, window_size=256, 
                                                                  feature_columns=feature_columns, label_features=label_features, 
                                                                  normalized_features=normalized_features)

## Load parameter, functions and dataloader

In [ ]:
tune_path = Path("/mnt/models/transformer/tune").absolute()
tune_path.mkdir(parents=True, exist_ok=True)

In [ ]:
from ray import tune, train as ray_train
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from torch import nn
from typing import Dict
from utils.cluster import attach_ray, disconnect_ray
from utils.optimizer import get_optimizer_function, get_learning_rate_scheduler
from utils.activation import get_activation
from utils.loss_functions import get_loss_function
from torch.utils.data import Dataset, DataLoader
import random; random.seed(0)

In [ ]:
num_samples = 2000
num_epochs = 750
grace_period = 5

In [ ]:
model_dim_params = []

n_neighbors = [3, 5, 7, 10, 15]
model_dim = [input_shape, output_shape]

for d in model_dim:
    num_heads = []
    for i in range(2, d + 1):
        if d % i == 0: num_heads.append(i)

    for h in num_heads:

        if d != output_shape:
            model_dim_params.append((d, h, 0))
        else:
            for n in n_neighbors:
                model_dim_params.append((d, h, n))

In [ ]:
start, end = np.log2(256), np.log2(2048)
num_values = 10
feedforward_dim = [int(2 ** (start + i / (num_values - 1) * (end - start))) for i in range(num_values)]

In [ ]:
parameter_space = {
    "pos_encoder_dropout": tune.loguniform(0.05, 0.6, base = 2),
    "transformer_dropout": tune.loguniform(0.05, 0.6, base = 2),
    "num_encoder_layer": tune.choice(list(range(2, 10 + 1, 2))),
    "feedforward_dim": tune.choice(feedforward_dim),
    "batch_size": tune.choice(list(range(64, 256, 16))),
    
    "model_dim_num_heads_projection": tune.choice(model_dim_params),
    "optimizer": tune.choice(["adam", "adamw"]),
    "activation": tune.choice(["relu", "gelu"]),
    "warmup_steps": tune.choice(list(range(1000, 4000, 200))),
}

In [ ]:
scheduler = ASHAScheduler(
    metric = "loss",
    mode = "min",
    max_t = num_epochs,
    grace_period = grace_period
)

In [ ]:
search_alg = OptunaSearch(
    metric = "loss",
    mode = "min"
) 

In [ ]:
attach_ray(manager = False)

In [ ]:
ray_resources_manager = tune.with_resources(
    trainable=lambda param: parameter_train(param, num_epochs, train_set, validation_set, input_shape, output_shape),
    # See: https://stackoverflow.com/questions/58967793/what-is-the-way-to-make-tune-run-parallel-trials-across-multiple-gpus
    resources={ "cpu": 3, "gpu": 0.25 if torch.cuda.is_available() else 0 }
)

tuner = tune.Tuner(
    ray_resources_manager,
    param_space=parameter_space,
    tune_config=tune.TuneConfig(
        scheduler=scheduler,
        search_alg=search_alg,
         num_samples=num_samples
    ),
    run_config = ray_train.RunConfig(
        name = "transformer_encoder"
    )
)

In [ ]:
results = tuner.fit()

In [ ]:
disconnect_ray()

In [ ]:
# Save as csv file
result_grid = results.get_dataframe()
result_grid.to_csv(tune_path / "trail_grid_.csv")

In [ ]:
best_result = result_grid.iloc[result_grid['loss'].idxmin()].to_dict()
trail_id = best_result['trial_id']

print(f"Trail ID from the best run: {trail_id}")

In [ ]:
print(f"Best trail by loss value {best_result['loss']}", "\n------")
for key in best_result:
    if 'config' in key:
        print(f"Best trail: {key} value {best_result[key]}")